---
**License**

 SNIaSimul

 Mon Jun 01 09:20:00 2020\
 Copyright  2020\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 SNIaSimul\
 Copyright (C) 2020 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
from chainconsumer import ChainConsumer



In [ ]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

cosmo = Nc.HICosmo.new_from_name (Nc.HICosmo, "NcHICosmoDEXcdm")

PF, Tgamma0_i = cosmo.param_index_from_name ("Tgamma0")
cosmo.param_set_lower_bound (Tgamma0_i, 0.0)

cosmo.props.H0      = 72.0
cosmo.props.Omegab  = 0.05
cosmo.props.Omegac  = 0.25
cosmo.props.Omegax  = 0.70
cosmo.props.Tgamma0 = 0.0
cosmo.props.w       = -1.0

cosmo.props.H0_fit     = True
cosmo.props.Omegac_fit = True
cosmo.props.Omegax_fit = True


In [ ]:
mset = Ncm.MSet ()
mset.set (cosmo)


In [ ]:
dist = Nc.Distance (zf = 2.5)

data = np.loadtxt ('DAT.txt')
snia_data = Nc.DataDistMu.new_empty (dist)
snia_data.set_size (len (data[:,0]))

offset = 19.3
data[:,1] = data[:,1] + offset

snia_data.x.set_data (data[:,0])
snia_data.y.set_data (data[:,1])
snia_data.sigma.set_all (0.4)
snia_data.props.w_mean = False

snia_data.set_init (True)

dset = Ncm.Dataset ()
dset.append_data (snia_data)

#snia_data.x.log_vals ("z: ", "% 22.15g", True)
#snia_data.y.log_vals ("y: ", "% 22.15g", True)
#snia_data.sigma.log_vals ("z: ", "% 22.15g", True)



In [ ]:
lh = Ncm.Likelihood (dataset = dset)
fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)
fit.run (Ncm.FitRunMsgs.SIMPLE)
#fit.run_restart (Ncm.FitRunMsgs.SIMPLE, 1.0e-4, 0.0, None, None)
fit.log_info ()


In [ ]:

plt.figure (figsize=(14, 7))
z_a = np.linspace (0.0, max (data[:,0]))
plt.plot (z_a, [dist.dmodulus (cosmo, z) + 5.0 * math.log10 (cosmo.RH_Mpc ()) for z in z_a])
plt.errorbar (data[:,0], data[:,1], yerr = [0.4] * len (data[:,1]), fmt= 'o')




In [ ]:
init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0e-3)

nwalkers = 50
stretch = Ncm.FitESMCMCWalkerStretch.new (nwalkers, mset.fparams_len ())

#stretch.set_scale (2.0)
stretch.set_box_mset (mset)

esmcmc  = Ncm.FitESMCMC.new (fit, nwalkers, init_sampler, stretch, Ncm.FitRunMsgs.SIMPLE)

esmcmc.set_auto_trim (True)
esmcmc.set_auto_trim_div (100)
esmcmc.set_max_runs_time (2.0 * 60.0)
#esmcmc.set_data_file ("SNiaSimul.fits")

In [ ]:
esmcmc.start_run ()
esmcmc.run_lre (501, 1.0e-2)
esmcmc.end_run ()

#esmcmc.peek_catalog ().trim_p (0.4)

esmcmc.mean_covar ()
fit.log_covar ()


In [ ]:
c = ChainConsumer ()

mcat = esmcmc.peek_catalog ()
nchains = mcat.nchains ()

rows   = np.array ([mcat.peek_row (i).dup_array () for i in range (mcat.len ())])
params = ["$" + mcat.col_symb (i) + "$" for i in range (mcat.ncols ())]
  
c.add_chain (rows[:,1:], walkers = nchains, parameters = params[1:], posterior = -rows[:,0]/2, name = "Test")
myparams = len (params[1:])


In [ ]:
fig = c.plotter.plot_walks (parameters = myparams, convolve = 100)


In [ ]:
fig = c.plotter.plot_distributions (parameters = myparams, figsize = 2.0)

In [ ]:
fig = c.plotter.plot (parameters = myparams, figsize = 3.0)